In [2]:
import numpy as np
import tensorflow as tf

In [3]:
def token_lookup():
    """
    Generate a dict to map punctuation into a token
    :return: dictionary mapping puncuation to token
    """
    return {
        '.': '||period||',
        ',': '||comma||',
        '"': '||quotes||',
        ';': '||semicolon||',
        '!': '||exclamation-mark||',
        '?': '||question-mark||',
        '(': '||left-parentheses||',
        ')': '||right-parentheses||',
        '--': '||emm-dash||',
        '\n': '||return||'
        
    }


In [4]:
num_epochs = 10000
batch_size = 512
rnn_size = 512
num_layers = 3
keep_prob = 0.7
embed_dim = 512
seq_length = 30
learning_rate = 0.001
save_dir = './save'

In [5]:
import tensorflow as tf
import numpy as np
import pickle

corpus_int, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))
seq_length, save_dir = pickle.load(open('params.p', mode='rb'))


In [6]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word with some randomness
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]


In [7]:
def generate_chapter(l, p):

    gen_length = l
    prime_words = p

    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load the saved model
        loader = tf.train.import_meta_graph(save_dir + '.meta')
        loader.restore(sess, save_dir)

        # Get tensors from loaded graph
        input_text = loaded_graph.get_tensor_by_name('input:0')
        initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
        final_state = loaded_graph.get_tensor_by_name('final_state:0')
        probs = loaded_graph.get_tensor_by_name('probs:0')

        # Sentences generation setup
        gen_sentences = prime_words.split()
        prev_state = sess.run(initial_state, {input_text: np.array([[1 for word in gen_sentences]])})

        # Generate sentences
        for n in range(gen_length):
            # Dynamic Input
            dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
            dyn_seq_length = len(dyn_input[0])

            # Get Prediction
            probabilities, prev_state = sess.run(
                [probs, final_state],
                {input_text: dyn_input, initial_state: prev_state})
            probabilities = probabilities[0]

            pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

            gen_sentences.append(pred_word)

        # Remove tokens
        chapter_text = ' '.join(gen_sentences)
        for key, token in token_dict.items():
            chapter_text = chapter_text.replace(' ' + token.lower(), key)
        
        chapter_text = ' '.join(gen_sentences)
        for key, token in token_dict.items():
            chapter_text = chapter_text.replace(' ' + token.lower(), key)
        chapter_text = chapter_text.replace('\n ', '\n')
        chapter_text = chapter_text.replace('( ', '(')
        chapter_text = chapter_text.replace(' ”', '”')

        capitalize_words = ['jesus','god','lord','christ','john','matthew','luke','mark',
                            'holy','ghost','moses','pharaoh','king','law','goliath','lebanon',
                            'horonaim','israel','jerusalem','levites','levite','david','ahab',
                            'joshua','esaias','ammon','jews','judah','babylon','benjamin',
                            'zion','i ','isaiah','amoz','abraham']

        for word in capitalize_words:
            chapter_text = chapter_text.replace(word, word.lower().title())
        
        return chapter_text

#         print(chapter_text)

In [11]:
chapter_text = generate_chapter(500, 'my')

chapter_text

INFO:tensorflow:Restoring parameters from ./save


"my Lord made the iniquity of the sin, and worship the word of God, and acceptable for the hope of these God. for he shall fly upon the Holy and, that I will not take a man in his ears, and shall rest. assemble of them, and come to watch of the King's word, and to live through the Lord: so he shall serve him in his sight. the judge in silver and the blood [shall be consecrated, and the crown of his brethren died, for the enemy taketh, even psalm armed her chariots in the camels, and [that] which die who fell at every time in that he knew him. and jonathan answered the Lord, saying, this [is] my rest, rightly whom my midwife said in a truth, ye know not these words, and God: I shall consume him in the dungeon; and there [are] all great world in the land of egypt effect by the imagination of their doings from sidon. behold, that passeth from every man of the rock, because [it is] a way for war, ye also shall bear an inheritance. o this man; ye shall sing to thy children from every provin

In [ ]:
import os
version_dir = './generated-book-v1'
if not os.path.exists(version_dir):
    os.makedirs(version_dir)

num_chapters = len([name for name in os.listdir(version_dir) if os.path.isfile(os.path.join(version_dir, name))])
next_chapter = version_dir + '/chapter-' + str(num_chapters + 1) + '.md'
with open(next_chapter, "w") as text_file:
    text_file.write(chapter_text)